<img src="./pics/DL.png" width=110 align="left" style="margin-right: 10px">

# Introduction to Deep Learning

## 09. Reinforcement Learning

---

## Reinforcement Learning

### Purpose

Reinforcement Learning(RL) is a type of machine learning technique that enables an agent to learn in an interactive environment by trial and error using feedback from its own actions and experiences.

Though both supervised and reinforcement learning use mapping between input and output, unlike supervised learning where feedback provided to the agent is correct set of actions for performing a task, reinforcement learning uses rewards and punishment as signals for positive and negative behavior.

As compared to unsupervised learning, reinforcement learning is different in terms of goals. While the goal in unsupervised learning is to find similarities and differences between data points, in reinforcement learning the goal is to find a suitable action model that would maximize the total cumulative reward of the agent. The figure below represents the basic idea and elements involved in a reinforcement learning model.

### How does it work?

<img src="./pics/external/rl/environment.jpg" alt="Reinforcement learning Environment - Agent">
<br>Image from <a href="https://www.kdnuggets.com/2018/03/5-things-reinforcement-learning.html">5 Things You Need to Know about Reinforcement Learning</a>, by <a href="https://www.kdnuggets.com/author/shweta-bhatt">Shweta Bhatt</a>, Youplus.

Main components of the Reinforcement Learning environment
- **Agent**: The controlled entity in the environment which can interact with the environment, execute actions and observe the rewards
- **Environment**: The abstract world in which the agent operates
- **State**: $S_t$ is the current state of the agent in the environment
- **Action**: $A_t$ is the interaction the agent selected to do in the current environment
- **Reward**: $R_t$ is the reward the agent receives from the environment as a response for its action. The goal of the agent is to maximize the received reward.
- **Policy**: $\pi \left(A_t = a | S_t = s, \theta \right)$ is the decision strategy of the agent. Based on the current state and the learned parameters the agent selects an action from its repertoire.
- **Value**: $V_{\pi}\left(S\right)$ The expected sum of rewards if the agent follows $\pi$ policy.

<img src="./pics/external/rl/mari-o.gif" alt="MarI/O">
<br>Image from <a href="https://www.youtube.com/watch?v=qv6UVOQ0F44">MarI/O - Machine Learning for Video Games</a>, by <a href="https://www.kdnuggets.com/author/shweta-bhatt">Seth Bling</a>.

The easiest way to get a grasp at Reinforcement Learning is through games: Using Super Mario World as an example, the agent's (aka. Mario's) goal is to get to the end of the level. The **world** around him **is the environment** where he must jump trough the enemies, (optionally) collect coins and kill enemies by jumping on them. Mario get's **rewards for progressing through the level** with a huge bonus when reaching the end but also got punishment (aka **negative reward**) **for walking into enemies**, or **falling into chasms** or letting the time run out. The **states are the position** of Mario **and the actual visible layout**; the **actions are the buttonpresses** on the controller.

At each timestamp the agent evaluates the possible actions considering the current state and based on the policy, it chooses the most beneficial action. Leraning an optimal policy requires the agent to **explore** the action space while **exploit** the incoming rewards - maximize the overall reward. This dual problem is called **exploration vs exploitation** tradoff.

The matematical frameworks to handle such problems are the **Markov Decision Process**es (MDP). With the help of MDPs, we can describe the environment in RL problems and formalize the process. MDPs describe the environment as a finite set of $S$ states, $A$ possible actions, $R$ reward values, and transitive possibilities $P(s', s | a)$. Real world environments does not contain prior knowledge about the environment - to handle this problem, RL has a set of processes called model-free approches.

Since Reinforcement Learning is a whole separate branch of the machine learning algorithms, we won't go deeper into the field, instead let's see how can we realize a specific model free learning method called **Q-learning**.

### Training

Two commonly used model-free algorithms are SARSA (State-Action-Reward-State-Action) and Q-learning. The main difference is how they explore their environment. SARSA is an on-policy method which learnd the overall value of an action by using the current policy to project its effectiveness. Q-learning is an off-policy algorithm, it uses an another (otherwise unused) policy to determine the value of an action. These methods are fairly simple but are not able to estimate values for unseen states.

To overcome this problem, we'll use the Deep Q-Networks algorithm to estimate Q-values. This algorithm is able to handle low-dimensional discrete action spaces. To handle continuous problems, Deep Deterministic Policy Gradients are a good starting point, however it is out of the scope of this discussion. 

Deep Q networks revolves around updating Q-values to estimate an action's value in a given state. The value update rule is the core of the Q-learning algorithm and can be written as:

$$\begin{align}
    Q^{\textrm{new}}\left( s_t, a_t \right) & =
    \underbrace{Q \left( s_t, a_t \right)}_\textrm{old value} + 
    \underbrace{\alpha}_\textrm{learning rate} \cdot
    \overbrace{\Bigg( 
        \underbrace{
            \underbrace{r_t}_\textrm{reward} +
            \underbrace{\gamma}_\textrm{discount factor} \cdot
            \underbrace{\max_{a}{Q\left( s_{t+1}, a \right)}}_\textrm{estimate of optimal future value}
        }_\textrm{new value (temporali difference target)} -
        \underbrace{Q\left( s_t, a_t \right)}_\textrm{old value}
    \Bigg)}^\textrm{temporal difference}
\end{align}$$

DQNs will learn online, meaning that we don’t simply create a bunch of trial/training data and feed it into the model, but we create training data through the trials we run and feed this information into it directly after running the trial instead.

### In Practice

DQNs are not part of any widespread deep learning library, so we'll have to implement the algoritm ourselves. To make it easier to understand the process let's consider a small (but not at all straightforward!) problem:

#### Building a hill-climber car

Since Reinforcement Learning living its renaissance there are frameworks to easily simulate the RL environment. One of the most popular framework is called gym. It contains the problem [Mountain Car](https://gym.openai.com/envs/MountainCar-v0/) where the goal is to drive up a car to a hill. The car in the problem does not have enough power to go up the hill, so the agent has to rely on building up a momentum. 

The state of the agent is the elevation of the car and the velocity; the available actions are accelerate to the left (0), don't accelerate (1), and accelerate to the right (2). The simulation ends if the car's elevation is above 0.5, or more than 200 episodes has passed.

First, let's see the problem using random actions in each episode:

In [ ]:
import gym

In [ ]:
class Environment:
    
    def __init__(self, problem):
        self.env = gym.make(problem)
        
    def __enter__(self):
        self.env.reset()
        return self.env
    
    def __exit__(self, *args, **kwargs):
        self.env.close()

#### I. Random solution

In [ ]:
with Environment('MountainCar-v0') as env:
    for episode in range(1000):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)

        print(f"Step {episode}:")
        print(f"action: {action}")
        print(f"observation: {observation}")
        print(f"reward: {reward}")
        print(f"done: {done}")
        print(f"info: {info}")

        if done:
            break

#### II. Naive solution

Build training data from random experiments, and train a NN on successful experiment samples.

In [ ]:
import random
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
def model_data_preparation(env, intial_games, episodes):
    env.reset()
    
    training_data = []
    accepted_scores = []
    for episode in range(intial_games):
        score = 0
        memory = []
        previous_state = []
        for episode in range(episodes):
            action = random.randrange(0, 3)
            state, reward, done, _ = env.step(action)
            
            if len(previous_state) > 0:
                memory.append([previous_state, action])
                
            previous_state = state
            if state[0] > -0.2:
                reward = 1
            
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in memory:
                state_memory, action_memory = data
                encoded_action = [0, 0, 0]
                encoded_action[action_memory] = 1
                training_data.append([state_memory, encoded_action])
        
        env.reset()
    
    print(accepted_scores)
    return training_data


def build_model(input_size, output_size):
    model = Sequential()
    
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    
    model.compile(loss='mse', optimizer=Adam())
    return model


def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    model.fit(X, y, epochs=10)
    
    return model

Generate train data and train model

In [ ]:
%%time
env = gym.make('MountainCar-v0')

episodes = 200
score_requirement = -198
intial_games = 10000

training_data = model_data_preparation(env, intial_games, episodes)
trained_model = train_model(training_data)

Test our trained model

In [ ]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    memory = []
    prev_state = []
    for episode in range(episodes):
        env.render()
        if len(prev_state) == 0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_state.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_state, reward, done, info = env.step(action)
        memory.append([new_state, action])
        
        prev_state = new_state
        score += reward
        
        if done:
            break

    env.reset()
    scores.append(score)

env.close()
print(scores)
print('Average Score:', sum(scores)/len(scores))
print('choice 1:{} choice 0:{} choice 2:{}'.format(choices.count(1) / len(choices),
                                                   choices.count(0) / len(choices),
                                                   choices.count(2) / len(choices)))

#### III. DQN solution

In [ ]:
import random

from collections import deque

import gym
import numpy as np

from tqdm.autonotebook import tqdm

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

A success counter helper class to track results of successful episodes.

In [ ]:
class Success:

    def __init__(self, threshold=10):
        self.sum = 0
        self.last10 = []
        self.last10sum = sum(self.last10)
        self.threshold = threshold
    
    def __iadd__(self, value):
        self.sum += value
        self.last10.append(value)
        self.last10 = self.last10[-10:]
        self.last10sum = sum(self.last10)
        return self

    def __add__(self, value):
        new = Success()
        new.sum = self.sum
        new.last10 = self.last10
        new.last10sum = self.last10sum
        new.threshold = self.threshold
        return new.__iadd__(value)

    def __bool__(self):
        return sum(self.last10) >= self.threshold

In [ ]:
class DQN:
    
    def __init__(self, env):
        self.env = env
        self.gamma = 0.99  # discount factor

        # control exploration vs explitation
        self.epsilon = 1
        self.epsilon_decay = 0.95
        self.epsilon_min = 0.01

        self.memory = deque(maxlen=20000)
        self.num_episodes = 400
        self.max_steps = 201  # max is 200
        
        # NeuralNet parameters
        self.learing_rate = 0.001
        self.batch_size = 32
        self.model = self.build_model()
        self.target_model = self.build_model()

        self.sync_models()

    def build_model(self):
        """Generate DNN to approximate Q-value.
        
        Create a network with:
        - 1 dense layer with 24 neurons using relu activation
        - 1 dense layer with 48 neurons using relu activation
        - 1 dense layer with action space neurons using linear activation
        """
        model = Sequential()
        state_shape = self.env.observation_space.shape
        action_shape = self.env.action_space.n

        model.add(Dense(24, activation='relu', input_shape=state_shape))
        model.add(Dense(48, activation='relu'))
        model.add(Dense(action_shape, activation='linear'))
        
        optimizer = Adam(learning_rate=self.learing_rate)
        model.compile(loss='mse', optimizer=optimizer)
        return model

    def sync_models(self):
        """Syncronize the learned and the off-policy model."""
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        """Select next action given the actual state."""
        # Exploration
        if np.random.rand(1) < self.epsilon:
            return np.random.randint(0, self.env.action_space.n)
        # Exploitation
        return np.argmax(self.model.predict(state)[0])

    def decay_epsilon(self):
        """Update epsilon value by decaying it until reaching minimum value."""
        if self.epsilon > self.epsilon_min:
            self.epsilon = max(self.epsilon_min, 
                               self.epsilon * self.epsilon_decay)

    def remember(self, state, action, reward, new_state, done):
        """Save episode results for later use."""
        self.memory.append([state, action, reward, new_state, done])

    def generate_batch(self):
        """Generate training batch from the saved memory by random sampling."""
        samples = np.array(random.sample(self.memory, self.batch_size))
        
        states, actions, rewards, new_states, dones = np.hsplit(samples, 5)

        states = np.concatenate(np.squeeze(states[:]), axis=0)    # [batch_size x 2]
        new_states = np.concatenate(np.concatenate(new_states))   # [batch_size x 2]
        rewards = rewards.reshape(self.batch_size,).astype(float) # [batch_size]
        actions = actions.reshape(self.batch_size,).astype(int)   # [batch_size]
        dones = dones.reshape(self.batch_size,).astype(bool)      # [batch_size]
        notdones = (~dones).astype(float)
    
        return states, actions, rewards, new_states, notdones

    def replay(self):
        """Updating Q values using the actual model and an off-policy model.
        
        The update rule is:
        Q(s_t, a_t) = Q(s_t, a_t) + alpha * (r_t + gamma * Q^*(s_t+1, a_t) - Q(s_t, a_t))
        
        Where the model is responsible of generating the Q(s_t, a_t) values, 
        and the off-policy model is responsible of generating the Q^*(s_t+1, a_t) values.
        
        The update is performed by the neural network backpropagation by setting the target
        value to r_t + gamma * Q^*(s_t+1, a_t).
        """
        if len(self.memory) < self.batch_size:
            return
        
        # I. Generating examples from memory
        states, actions, rewards, new_states, notdones = self.generate_batch()
        # II. Generating Q(s_t, a_t)
        targets = self.model.predict(states)
        indices = np.arange(self.batch_size)
        
        # III. Using off-policy model to predict future Q values: 
        # Q^*(s_t+1, a_t)
        Q_futures = self.target_model.predict(new_states).max(axis = 1)
        
        # IV. Generating temporal difference target
        # td = r_t + gamma * Q^*(s_t+1, a_t)
        targets[(indices, actions)] = rewards + self.gamma * Q_futures * notdones
        
        # V. Updating Q(s_t, a_t) by executing 1 training step
        self.model.fit(states, targets, epochs=1, verbose=0)

    def optimize_model(self, state, eps, render=True):
        score = 0
        max_position = -99

        for step in range(self.max_steps):
            # Show the animation every 50 eps
            if render and eps % 50 == 0:
                env.render()

            # select action
            action = self.act(state)
            # observe environment after taking action
            new_state, reward, done, _ = env.step(action)
            new_state = new_state.reshape(1, 2)

            # Keep track of max position
            position = new_state[0][0]
            if position > max_position:
                max_position = position

            # Adjust reward for task completion
            if position >= 0.5:
                reward += 10
            
            # Save episode results
            self.remember(state, action, reward, new_state, done)
            # Train network on observed events
            self.replay()
            
            # Update state and value reward
            state = new_state
            score += reward

            if done:
                break

        self.sync_models()
        self.decay_epsilon()
        
        return step < 199

    def fit(self, render=True):
        success = Success()
        episodes = tqdm(range(self.num_episodes))
        for episode in episodes:
            state = env.reset().reshape(1, 2)
            success += self.optimize_model(state, episode, render)

            episodes.set_postfix_str(f'overall: {success.sum}, '
                                     f'last10: {success.last10sum}')
            if success:
                print(f'10 success in a row, stopping early at episode {episode}.')
                episodes.close()
                break
        
        return self

                
def play(env, model, n=1):
    for _ in range(n):
        done = False
        state = env.reset().reshape(1, 2)
        while not done:
            env.render()
            action = model.act(state)
            new_state, reward, done, info = env.step(action)
            state = new_state.reshape(1, 2)

In [ ]:
with Environment('MountainCar-v0') as env:
    env.seed(42)
    random.seed(42)
    np.random.seed(42)

    dqn = DQN(env=env).fit(render=False)

In [ ]:
with Environment('MountainCar-v0') as env:
    play(env, dqn, 10)

## Exercise

### Solve the [CartPole](https://gym.openai.com/envs/CartPole-v1/) problem

Use the DQN algorithm to learn how to balance a pole on top of a moving cart. This is actually an easier task, since our action space is reduced greatYou have to modify the previously created class slightly. Can you make the class more general?

### Good job!

Thank you for your attention! Consider solving the exercises from the [DL 10 Home Assignment II.](./DL_10_Home_Assignment_II.ipynb) notebook.